<a href="https://colab.research.google.com/github/thakyemez/crime_hotspot_prediction/blob/main/Incorporating_SSRS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**PREPARING THE DATASETS**

**Importing the libraries**

In [ ]:
import numpy as np
import pandas as pd
import pickle
import networkx as nx
import scipy.sparse as sp
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
import argparse
import pickle
import os
import scipy.sparse as sp
import torch
from scipy.sparse import linalg
import torch.nn as nn
from torch.nn import BatchNorm2d, Conv1d, Conv2d, ModuleList, Parameter
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import os
from glob import glob
import matplotlib.pyplot as plt
import time
from fastprogress import progress_bar
import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
!pip install durbango
!pip install funcy
!pip install py3nvml
from durbango import pickle_save
!pip install geopandas

  Created wheel for durbango: filename=durbango-0.23-py3-none-any.whl size=31553 sha256=4df290b41cda002885912c3847ca1233ee7d70f1b4d2c67acc59e140c3996abe
  Stored in directory: /root/.cache/pip/wheels/4d/ab/8e/d7b59feb6ed6eaa5320c3e05700ec20fc62e0c04fddda07501
Successfully built durbango
     |████████████████████████████████| 55 kB 3.3 MB/s 


/usr/local/lib/python3.7/dist-packages/durbango/pd_utils.py:12: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


     |████████████████████████████████| 1.0 MB 14.4 MB/s 
     |████████████████████████████████| 15.4 MB 85.2 MB/s 
     |████████████████████████████████| 6.3 MB 73.6 MB/s 


**Preparing the Crime Dataset**

In [ ]:
##Theft
TheftTable = pd.read_csv('/content/gdrive/MyDrive/MainFiles/CenterSide/Transactions/ CenterTheftTable.csv')
a= pd.date_range(start ='1-1-2016 00:00:00',end ='31-12-2017 08:00:00', freq ='8H')
TheftTable.index=a
#TheftTable=TheftTable.drop(columns=['Time_ID'])
TheftTable=TheftTable.drop(columns=['Unnamed: 0'])
TheftTable_Smoothed=TheftTable.ewm(alpha=0.1).mean()
##Aggregating at day level
TheftTable_daily = TheftTable.resample('D').sum()
TheftTable_daily.sum()
TheftTable_daily.head()
TheftTable_daily.to_csv('/content/gdrive/MyDrive/transactions.csv')
TheftTable_daily_Smoothed=TheftTable_daily.ewm(alpha=0.05).mean()
TheftTable_daily_Smoothed.head()
##Robbery
RobberyTable = pd.read_csv('/content/gdrive/MyDrive/MainFiles/CenterSide/Transactions/ CenterRobberyTable.csv')
a= pd.date_range(start ='1-1-2016 00:00:00',end ='31-12-2017 08:00:00', freq ='8H')
RobberyTable.index=a
RobberyTable=RobberyTable.drop(columns=['Time_ID'])
RobberyTable=RobberyTable.drop(columns=['Unnamed: 0'])
RobberyTable_Smoothed=RobberyTable.ewm(alpha=0.1).mean()
##Aggregating at day level
RobberyTable_daily = RobberyTable.resample('D').sum()
RobberyTable_daily.sum()
RobberyTable_daily.head()
RobberyTable_daily.to_csv('/content/gdrive/MyDrive/transactions.csv')
RobberyTable_daily_Smoothed=RobberyTable_daily.ewm(alpha=0.05).mean()
RobberyTable_daily_Smoothed.head()

**Preparing the adjancency matrices for CS network**

In [ ]:
##Preparing the Center graph
import glob
paths=("/content/gdrive/MyDrive/MainFiles/CenterSide/Graphs")
all_files = glob.glob(paths + "/*.pickle")
for m in all_files:
  Cgraph=pd.read_pickle(m)
  nodescaC=np.array(TheftTable_daily_Smoothed.columns)
  nodescaC_to_ind=np.arange(0,2459,1)
  ntemp=nx.convert_matrix.to_numpy_matrix(Cgraph,nodelist=nodescaC,weight='weight')
  name=str(m.split('/')[-1]).split('.')[0]
  ext= '.pkl'
  with open("/content/gdrive/MyDrive/MainFiles/CenterSide/AdjMatrix/%s%s" %(name,ext),'wb') as f:
    pickle.dump([nodescaC,nodescaC_to_ind,ntemp], f)

**GENERATE TRAINING DATA**

In [ ]:
def generate_graph_seq2seq_io_data(
        df, x_offsets, y_offsets, add_time_in_day=True, add_day_in_week=False, scaler=None
):
    """
    Generate samples from
    :param df:
    :param x_offsets:
    :param y_offsets:
    :param add_time_in_day:
    :param add_day_in_week:
    :param scaler:
    :return:
    # x: (epoch_size, input_length, num_nodes, input_dim)
    # y: (epoch_size, output_length, num_nodes, output_dim)
    """

    num_samples, num_nodes = df.shape
    data = np.expand_dims(df.values, axis=-1)
    print("shape_of_data: ", data.shape)
    feature_list = [data]

    if add_time_in_day:
        time_ind = (df.index.values - df.index.values.astype("datetime64[D]")) / np.timedelta64(1, "D")
        time_in_day = np.tile(time_ind, [1, num_nodes, 1]).transpose((2, 1, 0))
        feature_list.append(time_in_day)
    if add_day_in_week:
        dow = df.index.dayofweek
        #Trying Weekend/Weekday distinction
        dow2=dow.copy()
        dow2=[1 if (x==5 or x==6) else 0  for x in dow2]
        dow=np.array(dow2)
        dow_tiled = np.tile(dow, [1, num_nodes, 1]).transpose((2, 1, 0))
        feature_list.append(dow_tiled)

    RobberyRSIS=RobberyRSIS_daily.values.reshape(num_samples, num_nodes,1)
    feature_list.append(RobberyRSIS)
    #Adding Normalized SSRS as an external variable
    RobberyRSIS=RobberyRSISS_normalized.values.reshape(num_samples, num_nodes,1)
    feature_list.append(RobberyRSIS)
    feature_list.append(RSISRobbery)
    data = np.concatenate(feature_list, axis=-1)
    print("shape_of_data: ", data.shape)
    x, y = [], []
    min_t = abs(min(x_offsets))
    print("min_t ", min_t)
    max_t = abs(num_samples - abs(max(y_offsets)))  # Exclusive
    print("max_t", max_t)
    for t in range(min_t, max_t):  # t is the index of the last observation. The loop iterates between 11 and 2180.
        x.append(data[t + x_offsets, ...])
        y.append(data[t + y_offsets, ...])
    print("x before stack ", x[0].shape)
    x = np.stack(x, axis=0)
    print("shape of x ", x.shape)
    print("y before stack ", y[0].shape)
    y = np.stack(y, axis=0)
    print("shape of y ", y.shape)
    return x, y

def generate_train_val_test(args):
    seq_length_x, seq_length_y = args.seq_length_x, args.seq_length_y
    df = args.traffic_df_filename
    # 0 is the latest observed sample.
    x_offsets = np.sort(np.concatenate((np.arange(-(seq_length_x - 1), 1, 1),)))
    print("shape of x_offsets ", x_offsets.shape)
    print("x_offsets ",x_offsets)
    # Predict the next one hour
    y_offsets = np.sort(np.arange(args.y_start, (seq_length_y + 1), 1))
    print("shape of y_offsets ", y_offsets.shape)
    print("y_offsets ",y_offsets)
    # x: (num_samples, input_length, num_nodes, input_dim)
    # y: (num_samples, output_length, num_nodes, output_dim)
    x, y = generate_graph_seq2seq_io_data(
        df,
        x_offsets=x_offsets,
        y_offsets=y_offsets,
        add_time_in_day=False,
        add_day_in_week=args.dow,
    )
    print("x shape: ", x.shape, ", y shape: ", y.shape)
    # Write the data into npz file.
    num_samples = x.shape[0]
    num_test = round(num_samples * 0.2)
    num_train = round(num_samples * 0.7)
    num_val = num_samples - num_test - num_train
    x_train, y_train = x[:num_train], y[:num_train]
    x_val, y_val = (
        x[num_train: num_train + num_val],
        y[num_train: num_train + num_val],
    )
    x_test, y_test = x[-num_test:], y[-num_test:]

    for cat in ["train", "val", "test"]:
        _x, _y = locals()["x_" + cat], locals()["y_" + cat]
        print(cat, "x: ", _x.shape, "y:", _y.shape)
        np.savez_compressed(
            os.path.join(args.output_dir, f"{cat}.npz"),
            x=_x,
            y=_y,
            x_offsets=x_offsets.reshape(list(x_offsets.shape) + [1]),
            y_offsets=y_offsets.reshape(list(y_offsets.shape) + [1]),
        )
    print("shape of x_offsets ", x_offsets.shape)
    print("shape of y_offsets ", y_offsets.shape)
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--output_dir", type=str, default="/content/gdrive/MyDrive/Robbery_Daily_data/", help="Output directory.")
    parser.add_argument("--traffic_df_filename", type=str, default=RobberyTable_daily_Smoothed, help=" Daily crime risk scores",)
    parser.add_argument("--seq_length_x", type=int, default=42, help="Sequence Length.",)
    parser.add_argument("--seq_length_y", type=int, default=1, help="Sequence Length.",)
    parser.add_argument("--y_start", type=int, default=1, help="Y pred start", )
    parser.add_argument("--dow", action='store_true',)

    args, unknown = parser.parse_known_args()
    if os.path.exists(args.output_dir):
        reply = str(input(f'{args.output_dir} exists. Do you want to overwrite it? (y/n)')).lower().strip()
        if reply[0] != 'y': exit
    else:
        os.makedirs(args.output_dir)
    generate_train_val_test(args)

/content/gdrive/MyDrive/Robbery_Daily_data/ exists. Do you want to overwrite it? (y/n)y
shape of x_offsets  (42,)
x_offsets  [-41 -40 -39 -38 -37 -36 -35 -34 -33 -32 -31 -30 -29 -28 -27 -26 -25 -24
 -23 -22 -21 -20 -19 -18 -17 -16 -15 -14 -13 -12 -11 -10  -9  -8  -7  -6
  -5  -4  -3  -2  -1   0]
shape of y_offsets  (1,)
y_offsets  [1]
shape_of_data:  (731, 2459, 1)
shape_of_data:  (731, 2459, 1)
min_t  41
max_t 730
x before stack  (42, 2459, 1)
shape of x  (689, 42, 2459, 1)
y before stack  (1, 2459, 1)
shape of y  (689, 1, 2459, 1)
x shape:  (689, 42, 2459, 1) , y shape:  (689, 1, 2459, 1)
train x:  (482, 42, 2459, 1) y: (482, 1, 2459, 1)
val x:  (69, 42, 2459, 1) y: (69, 1, 2459, 1)
test x:  (138, 42, 2459, 1) y: (138, 1, 2459, 1)
shape of x_offsets  (42,)
shape of y_offsets  (1,)


**TRAINING TESTING**

In [ ]:
hdir="/content/gdrive/MyDrive/"
best_model_save_path = os.path.join(hdir, 'best_model.pth')
def main(args, **model_kwargs):
    device = torch.device(args.device)
    data = load_dataset(args.data, args.batch_size, args.batch_size, args.batch_size, n_obs=args.n_obs, fill_zeroes=args.fill_zeroes)
    scaler = data['scaler']
    aptinit, supports = make_graph_inputs(args, device)
    torch.manual_seed(7)
    model = GWNet.from_args(args, device, supports, aptinit, **model_kwargs)
    if args.checkpoint:

      model.load_checkpoint(torch.load(args.checkpoint))
    model.to(device)
    engine = Trainer.from_args(model, scaler, args)
    metrics = []
    best_model_save_path = os.path.join(hdir, 'best_model.pth')
    lowest_mae_yet = 100  # high value, will get overwritten
    mb = progress_bar(list(range(1, args.epochs + 1)))
    epochs_since_best_mae = 0
    for _ in mb:
        train_loss, train_mape, train_rmse = [], [], []
        data['train_loader'].shuffle()
        for iter, (x, y) in enumerate(data['train_loader'].get_iterator()):
          trainx = torch.Tensor(x).to(device).transpose(1, 3)
          trainy = torch.Tensor(y).to(device).transpose(1, 3)
          yspeed = trainy[:, 0, :, :]
          if yspeed.max() == 0: continue
              #print('traınx',trainx.shape)
              #print('traıny',trainy.shape)
              #print('yspeed',yspeed.shape)
          mae, mape, rmse = engine.train(trainx, yspeed)
          train_loss.append(mae)
          train_mape.append(mape)
          train_rmse.append(rmse)
          if args.n_iters is not None and iter >= args.n_iters:
              break
        engine.scheduler.step()
        _, valid_loss, valid_mape, valid_rmse = eval_(data['val_loader'], device, engine)
        m = dict(train_loss=np.mean(train_loss), train_mape=np.mean(train_mape),
                      train_rmse=np.mean(train_rmse), valid_loss=np.mean(valid_loss),
                        valid_mape=np.mean(valid_mape), valid_rmse=np.mean(valid_rmse))

        m = pd.Series(m)
        metrics.append(m)
        if m.valid_loss < lowest_mae_yet:
            torch.save(engine.model.state_dict(), best_model_save_path)
            lowest_mae_yet = m.valid_loss
            epochs_since_best_mae = 0
        else:
            epochs_since_best_mae += 1
        met_df = pd.DataFrame(metrics)
        mb.comment = f'best val_loss: {met_df.valid_loss.min(): .3f}, current val_loss: {m.valid_loss:.3f}, current train loss: {m.train_loss: .3f}'
        met_df.round(6).to_csv(f'{hdir}/metrics.csv')
        if epochs_since_best_mae >= args.es_patience: break
        # Metrics on test data
    with open(best_model_save_path, 'rb') as f:
      buffer = io.BytesIO(f.read())
    engine.model.load_state_dict(torch.load(buffer))
    realy = torch.Tensor(data['y_test']).transpose(1, 3)[:, 0, :, :].to(device)
    test_met_df, yhat = calc_tstep_metrics(engine.model, device, data['test_loader'], scaler, realy, args.seq_length)
    test_met_df.round(12).to_csv(os.path.join(hdir, 'test_metrics.csv'))

def eval_(ds, device, engine):
    """Run validation."""
    valid_loss = []
    valid_mape = []
    valid_rmse = []
    s1 = time.time()
    for (x, y) in ds.get_iterator():
        testx = torch.Tensor(x).to(device).transpose(1, 3)
        testy = torch.Tensor(y).to(device).transpose(1, 3)
        metrics = engine.eval(testx, testy[:, 0, :, :])
        valid_loss.append(metrics[0])
        valid_mape.append(metrics[1])
        valid_rmse.append(metrics[2])
    total_time = time.time() - s1
    return total_time, valid_loss, valid_mape, valid_rmse


if __name__ == "__main__":
    parser =get_shared_arg_parser()
    parser.add_argument('--epochs', type=int, default=100, help='')
    parser.add_argument('--clip', type=int, default=3, help='Gradient Clipping')
    parser.add_argument('--weight_decay', type=float, default=0.005, help='weight decay rate')
    parser.add_argument('--learning_rate', type=float, default=0.05, help='learning rate')
    parser.add_argument('--lr_decay_rate', type=float, default=0.935, help='learning rate')
    parser.add_argument('--save', type=str, default='Robbery_Daily_experiment', help='save path')
    parser.add_argument('--n_iters', default=None, help='quit after this many iterations')
    parser.add_argument('--es_patience', type=int, default=20, help='quit if no improvement after this many iterations')

    args, unknown = parser.parse_known_args()
    t1 = time.time()
    if not os.path.exists(hdir):
        os.mkdir(hdir)
    pickle_save(args,'/content/gdrive/MyDrive/args.pkl')
    main(args)
    t2 = time.time()
    mins = (t2 - t1) / 60
    print(f"Total time spent: {mins:.2f} seconds")
    #pref=m.split('/')[-1]
    #pp=p.split('/')[-1].split('.')[0].split('graph')[0].split('Center')[1]
    #key=str("experiment:"+"-"+ str(k)+"-"+str(l)+"-"+str(n)+".csv")
    key=str("Robbery_Daily_experiment:"+".csv")

    epath=('/content/gdrive/MyDrive/Robbery_Daily_experiment/%s'%(key))
def main(args, save_pred_path=epath, save_metrics_path='last_test_metrics_Robbery_preds_10day_dayca8_smoothingcoef05.csv', loader='test', **model_kwargs):
    args.checkpoint= best_model_save_path
    device = torch.device(args.device)
    adjinit, supports = make_graph_inputs(args, device)
    model = GWNet.from_args(args, device, supports, adjinit,**model_kwargs)
    model.load_state_dict(torch.load(args.checkpoint))
    model.to(device)
    model.eval()
    print('model loaded successfully')
    data = load_dataset(args.data, args.batch_size, args.batch_size, args.batch_size, n_obs=args.n_obs, fill_zeroes=args.fill_zeroes)
    scaler = data['scaler']
    realy = torch.Tensor(data[f'y_{loader}']).to(device)
    realy = realy.transpose(1,3)[:,0,:,:]
    met_df, yhat = calc_tstep_metrics(model, device, data[f'{loader}_loader'], scaler, realy, args.seq_length)
    df2 = make_pred_df(realy, yhat, scaler, args.seq_length)
    met_df.to_csv(save_metrics_path)
    df2.to_csv(save_pred_path, index=False)
    if args.plotheatmap:
      plot_learned_adj_matrix(model)
    return met_df, df2

def plot_learned_adj_matrix(model):
    adp = F.softmax(F.relu(torch.mm(model.nodevec1, model.nodevec2)), dim=1)
    adp = adp.cpu().detach().numpy()
    adp = adp / np.max(adp)
    df = pd.DataFrame(adp)
    sns.heatmap(df, cmap="RdYlBu")
    plt.savefig("/content/gdrive/MyDrive/heatmap.png")


if __name__ == "__main__":
    parser = get_shared_arg_parser()
    parser.add_argument('--checkpoints', type=str, help='')
    parser.add_argument('--plotheatmap', action='store_true')
    args, unknown = parser.parse_known_args()
    main(args)

Total time spent: 4.38 seconds
model loaded successfully


**UTIL**

In [ ]:
DEFAULT_DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

class DataLoader(object):
    def __init__(self, xs, ys, batch_size, pad_with_last_sample=True):
        """
        :param xs:
        :param ys:
        :param batch_size:
        :param pad_with_last_sample: pad with the last sample to make number of samples divisible to batch_size.
        """
        self.batch_size = batch_size
        self.current_ind = 0
        if pad_with_last_sample:
            num_padding = (batch_size - (len(xs) % batch_size)) % batch_size
            x_padding = np.repeat(xs[-1:], num_padding, axis=0)
            y_padding = np.repeat(ys[-1:], num_padding, axis=0)
            xs = np.concatenate([xs, x_padding], axis=0)
            ys = np.concatenate([ys, y_padding], axis=0)
        self.size = len(xs)
        self.num_batch = int(self.size // self.batch_size)
        #print("number of batches",self.num_batch)
        self.xs = xs
        self.ys = ys

    def shuffle(self):
        permutation = np.random.permutation(self.size)
        xs, ys = self.xs[permutation], self.ys[permutation]
        self.xs = xs
        self.ys = ys

    def get_iterator(self):
        self.current_ind = 0

        def _wrapper():
            while self.current_ind < self.num_batch:
                start_ind = self.batch_size * self.current_ind
                end_ind = min(self.size, self.batch_size * (self.current_ind + 1))
                x_i = self.xs[start_ind: end_ind, ...]
                y_i = self.ys[start_ind: end_ind, ...]
                yield (x_i, y_i)
                self.current_ind += 1

        return _wrapper()


class StandardScaler():

    def __init__(self, mean, std, fill_zeroes=True):
        self.mean = mean
        self.std = std
        self.fill_zeroes = fill_zeroes

    def transform(self, data):
        if self.fill_zeroes:
            mask = (data == 0)
            data[mask] = self.mean
        return (data - self.mean) / self.std

    def inverse_transform(self, data):
        return (data * self.std) + self.mean



def sym_adj(adj):
    """Symmetrically normalize adjacency matrix."""
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).astype(np.float32).todense()

def asym_adj(adj):
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1)).flatten()
    d_inv = np.power(rowsum, -1).flatten()
    d_inv[np.isinf(d_inv)] = 0.
    d_mat= sp.diags(d_inv)
    return d_mat.dot(adj).astype(np.float32).todense()

def calculate_normalized_laplacian(adj):
    """
    # L = D^-1/2 (D-A) D^-1/2 = I - D^-1/2 A D^-1/2
    # D = diag(A 1)
    :param adj:
    :return:
    """
    adj = sp.coo_matrix(adj)
    d = np.array(adj.sum(1))
    d_inv_sqrt = np.power(d, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    normalized_laplacian = sp.eye(adj.shape[0]) - adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()
    return normalized_laplacian

def calculate_scaled_laplacian(adj_mx, lambda_max=2, undirected=True):
    if undirected:
        adj_mx = np.maximum.reduce([adj_mx, adj_mx.T])
    L = calculate_normalized_laplacian(adj_mx)
    if lambda_max is None:
        lambda_max, _ = linalg.eigsh(L, 1, which='LM')
        lambda_max = lambda_max[0]
    L = sp.csr_matrix(L)
    M, _ = L.shape
    I = sp.identity(M, format='csr', dtype=L.dtype)
    L = (2 / lambda_max * L) - I
    return L.astype(np.float32).todense()

def load_pickle(pickle_file):
    try:
        with open(pickle_file, 'rb') as f:
            pickle_data = pickle.load(f)
    except UnicodeDecodeError as e:
        with open(pickle_file, 'rb') as f:
            pickle_data = pickle.load(f, encoding='latin1')
    except Exception as e:
        #print('Unable to load data ', pickle_file, ':', e)
        raise
    return pickle_data

ADJ_CHOICES = ['scalap', 'normlap', 'symnadj', 'transition', 'identity']
def load_adj(pkl_filename, adjtype):
    sensor_ids, sensor_id_to_ind, adj_mx = load_pickle(pkl_filename)
    if adjtype == "scalap":
        adj = [calculate_scaled_laplacian(adj_mx)]
    elif adjtype == "normlap":
        adj = [calculate_normalized_laplacian(adj_mx).astype(np.float32).todense()]
    elif adjtype == "symnadj":
        #print('sym')
        adj = [sym_adj(adj_mx)]
    elif adjtype == "transition":
        adj = [asym_adj(adj_mx)]
    elif adjtype == "doubletransition":
        adj = [asym_adj(adj_mx), asym_adj(np.transpose(adj_mx))]
    elif adjtype == "identity":
        adj = [np.diag(np.ones(adj_mx.shape[0])).astype(np.float32)]
    else:
        error = 0
        assert error, "adj type not defined"
    #print("sensor_ids",sensor_ids)
    #print("sensor_id_to_ind",sensor_id_to_ind)
    return sensor_ids, sensor_id_to_ind, adj


def load_dataset(dataset_dir, batch_size, valid_batch_size=None, test_batch_size=None, n_obs=None, fill_zeroes=True):
    data = {}
    for category in ['train', 'val', 'test']:
        cat_data = np.load(os.path.join(dataset_dir, category + '.npz'))
        data['x_' + category] = cat_data['x']
        data['y_' + category] = cat_data['y']
        if n_obs is not None:
            data['x_' + category] = data['x_' + category][:n_obs]
            data['y_' + category] = data['y_' + category][:n_obs]
    scaler = StandardScaler(mean=data['x_train'][..., 0].mean(), std=data['x_train'][..., 0].std(), fill_zeroes=fill_zeroes)
    # Data format
    for category in ['train', 'val', 'test']:
        data['x_' + category][..., 0] = scaler.transform(data['x_' + category][..., 0])
    data['train_loader'] = DataLoader(data['x_train'], data['y_train'], batch_size)
    data['val_loader'] = DataLoader(data['x_val'], data['y_val'], valid_batch_size)
    data['test_loader'] = DataLoader(data['x_test'], data['y_test'], test_batch_size)
    data['scaler'] = scaler
    return data


def calc_metrics(preds, labels,device='cuda:0', null_val=0.):
    if np.isnan(null_val):
        mask = ~torch.isnan(labels)
    else:
        mask = (labels != null_val)
    mask = mask.float()
    mask /= torch.mean(mask)
    mask = torch.where(torch.isnan(mask), torch.zeros_like(mask), mask)
    #print("mask shape",mask.shape)
    mse = (preds - labels) ** 2
    #print("preds.shape",preds.shape)
    #print("labels.shape",preds.shape)
    #print("mse shape",mse.shape)
    #print("mse",mse)
    #mae = (torch.abs(preds-labels)) # original mae
    #print("mae shape before mask",mae.shape)
    #print("mae type before mask", type(mae.shape))
    ##adapted mae
    if preds.shape[0]==args.batch_size:
      fpreds=torch.flatten(preds).to(device).cpu().data.numpy()
      flabels=torch.flatten(labels).to(device).cpu().data.numpy()
      #print("flabels.shape",fpreds.shape)
      #c=[abs(x-y)*y if y >0 else abs(x-y)*(10**-100)  for x,y in zip(fpreds,flabels)]
      ##Trying MSE  ##Trying MSE yielded inferior performance
      ##c=[((x-y)**2)*y if y >0 else ((x-y)**2)*(10**-100)  for x,y in zip(fpreds,flabels)]
      ##Quantile loss
      c=[((0.95-1)*((x-y)**2)) if y==0 else ((0.95)*((x-y)**2)) for x,y in zip(fpreds,flabels)]
      a=np.array(c)
      b=a.reshape(-1,2459,1)
      mae=torch.tensor([b],requires_grad=True).to(device)
    else:
      d=preds.shape[0]
      fpreds=torch.flatten(preds).to(device).cpu().data.numpy()
      flabels=torch.flatten(labels).to(device).cpu().data.numpy()
      #print("flabels.shape",fpreds.shape)
      #c=[abs(x-y)*y if y >0 else abs(x-y)*(10**-100)  for x,y in zip(fpreds,flabels)]
      ##Trying MSE yielded inferior performance
      #c=[((x-y)**2)*y if y >0 else ((x-y)**2)*(10**-100)  for x,y in zip(fpreds,flabels)]
      ##Quantile loss
      c=[((0.95-1)*((x-y)**2)) if y==0 else ((0.95)*((x-y)**2)) for x,y in zip(fpreds,flabels)]
      a=np.array(c)
      b=a.reshape(d,2459)
      mae=torch.tensor([b],requires_grad=True).to(device)
    #print("mae train",mae)
    #print("preds shape in calc_metrics",preds.shape)
    #print("labels shape in calc_metrics",labels.shape)
    mape = mae / labels
    #print("mae before mask",mae)
    #print("mae mean before mask",mae.mean)
    mae, mape, mse = [mask_and_fillna(l, mask) for l in [mae, mape, mse]]
    #print("mae after mask",mae)
    rmse = torch.sqrt(mse)
    return mae, mape, rmse


def mask_and_fillna(loss, mask):
    loss = loss * mask
    #print("shape of loss", loss.shape)
    #print("shape of mask", mask.shape)
    loss = torch.where(torch.isnan(loss), torch.zeros_like(loss), loss)
    return torch.mean(loss)


def calc_tstep_metrics(model, device, test_loader, scaler, realy, seq_length) -> pd.DataFrame:
    model.eval()
    outputs = []
    for _, (x, __) in enumerate(test_loader.get_iterator()):
        testx = torch.Tensor(x).to(device).transpose(1, 3)
        #print("testx.shape",testx.shape)
        with torch.no_grad():
            preds = model(testx).transpose(1, 3)
            #print("preds.shape",preds.shape)
        outputs.append(preds.squeeze(1))
    yhat = torch.cat(outputs, dim=0)[:realy.size(0), ...]
    #print("yhat.shape in calc tstep metrics",yhat.shape)
    test_met = []

    for i in range(seq_length):
        pred = scaler.inverse_transform(yhat[:, :, i])
        pred = torch.clamp(pred, min=0., max=70.)
        real = realy[:, :, i]
        test_met.append([x.item() for x in calc_metrics(pred, real)])
    test_met_df = pd.DataFrame(test_met, columns=['mae', 'mape', 'rmse']).rename_axis('t')
    #print("test_met_df.shape",test_met_df.shape)
    return test_met_df, yhat


def _to_ser(arr):
    return pd.DataFrame(arr.cpu().detach().numpy()).stack().rename_axis(['obs', 'sensor_id'])


def make_pred_df(realy, yhat, scaler, seq_length):
    #print("realy.shape",realy.shape)
    #print("yhat.shape",yhat.shape)
    df = pd.DataFrame(dict(y_last=_to_ser(realy[:, :, seq_length - 1]),
                           yhat_last=_to_ser(scaler.inverse_transform(yhat[:, :, seq_length - 1]))))
                           #y_3=_to_ser(realy[:, :, 2]),
                           #yhat_3=_to_ser(scaler.inverse_transform(yhat[:, :, 2]))))
    return df


def make_graph_inputs(args, device):
    sensor_ids, sensor_id_to_ind, adj_mx = load_adj(args.adjdata, args.adjtype)
    supports = [torch.tensor(i).to(device) for i in adj_mx]
    aptinit = None if args.randomadj else supports[0]  # ignored without do_graph_conv and add_apt_adj
    if args.aptonly:
        if not args.addaptadj and args.do_graph_conv: raise ValueError(
            'WARNING: not using adjacency matrix')
        supports = None
    return aptinit, supports
def get_shared_arg_parser():
    parser = argparse.ArgumentParser()
    parser.add_argument('--device', type=str, default='cuda:0' if torch.cuda.is_available() else 'cpu', help='')
    parser.add_argument('--data', type=str, default="/content/gdrive/MyDrive/Robbery_Daily_data", help='data path')
    parser.add_argument('--adjdata', type=str, default='/content/gdrive/MyDrive/MainFiles/CenterSide/AdjMatrix/Center01graph.pkl',help='adj data path')
    ##for grid search
    #parser.add_argument('--data', type=str, default=m, help='data path')
    #parser.add_argument('--adjdata', type=str, default=p,help='adj data path')
    parser.add_argument('--adjtype', type=str, default='symnadj', help='adj type', choices=ADJ_CHOICES)
    parser.add_argument('--do_graph_conv', action='store_true',
                        help='whether to add graph convolution layer')
    parser.add_argument('--aptonly', default=False, help='whether only adaptive adj')
    parser.add_argument('--addaptadj', action='store_true', help='whether add adaptive adj')
    parser.add_argument('--randomadj', action='store_true',
                        help='whether random initialize adaptive adj')
    parser.add_argument('--seq_length', type=int, default=1, help='')
    parser.add_argument('--nhid', type=int, default=8, help='Number of channels for internal conv')
    parser.add_argument('--in_dim', type=int, default=1, help='inputs dimension')
    parser.add_argument('--num_nodes', type=int, default=2459, help='number of nodes')
    parser.add_argument('--batch_size', type=int, default=16, help='batch size')
    parser.add_argument('--dropout', type=float, default=0.3, help='dropout rate')
    parser.add_argument('--n_obs', default=None, help='Only use this many observations. For unit testing.')
    parser.add_argument('--apt_size', default=10, type=int)
    parser.add_argument('--cat_feat_gc', action='store_true')
    parser.add_argument('--fill_zeroes', action='store_true')
    parser.add_argument('--checkpoint', type=str, help='')
    return parser

**MODEL**

In [ ]:
def nconv(x, A):
    """Multiply x by adjacency matrix along source node axis"""
    return torch.einsum('ncvl,vw->ncwl', (x, A)).contiguous()

class GraphConvNet(nn.Module):
    def __init__(self, c_in, c_out, dropout, support_len=3, order=2):
        super().__init__()
        c_in = (order * support_len + 1) * c_in
        self.final_conv = Conv2d(c_in, c_out, (1, 1), padding=(0, 0), stride=(1, 1), bias=True)
        self.dropout = dropout
        self.order = order

    def forward(self, x, support: list):
        out = [x]
        #print("x shape in forward",x.shape)
        for a in support:
            x1 = nconv(x, a)
            out.append(x1)
            for k in range(2, self.order + 1):
                x2 = nconv(x1, a)
                out.append(x2)
                x1 = x2

        h = torch.cat(out, dim=1)
        h = self.final_conv(h)
        h = F.dropout(h, self.dropout, training=self.training)
        #print("hidden layer shape",h.shape)
        return h


class GWNet(nn.Module):
    def __init__(self, device, num_nodes,supports, dropout=0.3, do_graph_conv=True,
                 addaptadj=True,aptinit=None, in_dim=1, out_dim=1,
                 residual_channels=32, dilation_channels=32, cat_feat_gc=False,
                 skip_channels=256, end_channels=512, kernel_size=2, blocks=7, layers=3,
                 apt_size=10): #here we had to delete =none statement for supports and aptinit argument
        super().__init__()
        self.dropout = dropout
        self.blocks = blocks
        self.layers = layers
        self.do_graph_conv = do_graph_conv
        self.cat_feat_gc = cat_feat_gc
        self.addaptadj = addaptadj


        if self.cat_feat_gc:
            self.start_conv = nn.Conv2d(in_channels=1,  # hard code to avoid errors
                                        out_channels=residual_channels,
                                        kernel_size=(1, 1))

            self.cat_feature_conv = nn.Conv2d(in_channels=in_dim - 1,
                                              out_channels=residual_channels,
                                              kernel_size=(1, 1))
        else:
            self.start_conv = nn.Conv2d(in_channels=in_dim,
                                        out_channels=residual_channels,
                                        kernel_size=(1, 1))

        self.fixed_supports = supports or []
        receptive_field = 1

        self.supports_len = len(self.fixed_supports)
        if do_graph_conv and addaptadj:
            if aptinit is None:
                #print("randomnodevecs")
                nodevecs = torch.randn(num_nodes, apt_size), torch.randn(apt_size, num_nodes)
            else:
                #print("not random vecs")
                nodevecs = self.svd_init(apt_size, aptinit)
            #print("shape nodevecs",nodevecs.shape)
            self.supports_len += 1
            self.nodevec1, self.nodevec2 = [Parameter(n.to(device), requires_grad=True) for n in nodevecs]

        depth = list(range(blocks * layers))

        # 1x1 convolution for residual and skip connections (slightly different see docstring)
        self.residual_convs = ModuleList([Conv1d(dilation_channels, residual_channels, (1, 1)) for _ in depth])
        self.skip_convs = ModuleList([Conv1d(dilation_channels, skip_channels, (1, 1)) for _ in depth])
        self.bn = ModuleList([BatchNorm2d(residual_channels) for _ in depth])
        self.graph_convs = ModuleList([GraphConvNet(dilation_channels, residual_channels, dropout, support_len=self.supports_len)
                                              for _ in depth])

        self.filter_convs = ModuleList()
        self.gate_convs = ModuleList()
        for b in range(blocks):
            additional_scope = kernel_size - 1
            D = 1 # dilation
            for i in range(layers):
                # dilated convolutions
                self.filter_convs.append(Conv2d(residual_channels, dilation_channels, (1, kernel_size), dilation=D))
                self.gate_convs.append(Conv1d(residual_channels, dilation_channels, (1, kernel_size), dilation=D))
                D *= 2
                receptive_field += additional_scope
                additional_scope *= 2
        self.receptive_field = receptive_field
        self.end_conv_1 = Conv2d(skip_channels, end_channels, (1, 1), bias=True)
        self.end_conv_2 = Conv2d(end_channels, out_dim, (1, 1), bias=True)

    @staticmethod
    def svd_init(apt_size, aptinit):
        m, p, n = torch.linalg.svd(aptinit)
        nodevec1 = torch.mm(m[:, :apt_size], torch.diag(p[:apt_size] ** 0.5))
        nodevec2 = torch.mm(torch.diag(p[:apt_size] ** 0.5), n[:, :apt_size].t())
        #print("nodevec1 shape",nodevec1.shape)
        #print("nodevec2 shape",nodevec2.shape)
        return nodevec1, nodevec2

    @classmethod
    def from_args(cls, args, device, supports, aptinit, **kwargs):
        defaults = dict(dropout=args.dropout, supports=supports,
                        do_graph_conv=args.do_graph_conv, addaptadj=args.addaptadj, aptinit=aptinit,
                        in_dim=args.in_dim, apt_size=args.apt_size, out_dim=args.seq_length,
                        residual_channels=args.nhid, dilation_channels=args.nhid,
                        skip_channels=args.nhid * 8, end_channels=args.nhid * 16,
                        cat_feat_gc=args.cat_feat_gc)
        defaults.update(**kwargs)
        model = cls(device, args.num_nodes, **defaults)
        return model

    def load_checkpoint(self, state_dict):
        """It is assumed that ckpt was trained to predict a subset of timesteps."""
        bk, wk = ['end_conv_2.bias', 'end_conv_2.weight']  # only weights that depend on seq_length
        b, w = state_dict.pop(bk), state_dict.pop(wk)
        self.load_state_dict(state_dict, strict=False)
        cur_state_dict = self.state_dict()
        cur_state_dict[bk][:b.shape[0]] = b
        cur_state_dict[wk][:w.shape[0]] = w
        self.load_state_dict(cur_state_dict)

    def forward(self, x):
        #print('forwardın bası', x.shape)
        # Input shape is (bs, features, n_nodes, n_timesteps)
        in_len = x.size(3)
        if in_len < self.receptive_field:
            x = nn.functional.pad(x, (self.receptive_field - in_len, 0, 0, 0))
            #print('if receptive_field',x.shape)
        if self.cat_feat_gc:
            f1, f2 = x[:, [0]], x[:, 1:]
            x1 = self.start_conv(f1)
            x2 = F.leaky_relu(self.cat_feature_conv(f2))
            x = x1 + x2
        else:
            x = self.start_conv(x)
            #print('if start conv',x.shape)
        skip = 0
        adjacency_matrices = self.fixed_supports
        # calculate the current adaptive adj matrix once per iteration
        if self.addaptadj:
            adp = F.softmax(F.relu(torch.mm(self.nodevec1, self.nodevec2)), dim=1)
            adjacency_matrices = self.fixed_supports + [adp]

        # WaveNet layers
        for i in range(self.blocks * self.layers):
            # EACH BLOCK

            #            |----------------------------------------|     *residual*
            #            |                                        |
            #            |   |-dil_conv -- tanh --|                |
            #         ---|                  * ----|-- 1x1 -- + -->	*x_in*
            #                |-dil_conv -- sigm --|    |
            #                                         1x1
            #                                          |
            # ---------------------------------------> + ------------->	*skip*
            residual = x
            # dilated convolution
            filter = torch.tanh(self.filter_convs[i](residual))
            gate = torch.sigmoid(self.gate_convs[i](residual))
            x = filter * gate
            #print('filter*gate',x.shape)
            # parametrized skip connection
            s = self.skip_convs[i](x)  # what are we skipping??
            try:  # if i > 0 this works
                skip = skip[:, :, :,  -s.size(3):]  # TODO(SS): Mean/Max Pool?
            except:
                skip = 0
            skip = s + skip
            if i == (self.blocks * self.layers - 1):  # last X getting ignored anyway
                break

            if self.do_graph_conv:
                graph_out = self.graph_convs[i](x, adjacency_matrices)
                x = x + graph_out if self.cat_feat_gc else graph_out
                #print('if do graph conv',x.shape)
            else:
                x = self.residual_convs[i](x)
                #print('if residual convs',x.shape)
            x = x + residual[:, :, :, -x.size(3):]  # TODO(SS): Mean/Max Pool?
            #print('x + residual[:, :, :, -x.size(3):]',x.shape)
            x = self.bn[i](x)
            #print('self.bn[i](x)',x.shape)

        x = F.relu(skip)  # ignore last X?
        x = F.relu(self.end_conv_1(x))
        #print('relu and end.conv1',x.shape)
        x = self.end_conv_2(x)  # downsample to (bs, seq_length, 1804, nfeatures)
        #print("x in forward",x.shape)
        return x

**ENGINE**

In [ ]:
class Trainer():
    def __init__(self, model: GWNet, scaler, lrate, wdecay, clip=3, lr_decay_rate=.97):
        self.model = model

        self.optimizer = optim.Adam(self.model.parameters(), lr=lrate, weight_decay=wdecay)
        self.scaler = scaler
        self.clip = clip
        self.scheduler = optim.lr_scheduler.LambdaLR(
            self.optimizer, lr_lambda=lambda epoch: lr_decay_rate ** epoch)

    @classmethod
    def from_args(cls, model, scaler, args):
        return cls(model, scaler, args.learning_rate, args.weight_decay, clip=args.clip,
                   lr_decay_rate=args.lr_decay_rate)

    def train(self, input, real_val):
       # print('engıne traın bası',input.shape)
        self.model.train()
        self.optimizer.zero_grad()
        input = nn.functional.pad(input,(1,0,0,0))
        #print("inputshape in engine.train",input.shape)
        output = self.model(input).transpose(1,3)  # now, output = [batch_size,1,num_nodes, seq_length]
        #print("outputshape in engine.train",output.shape)
        predict = self.scaler.inverse_transform(output)
        #print("predictshape in engine.train",predict.shape)
        assert predict.shape[1] == 1
        mae, mape, rmse = calc_metrics(predict.squeeze(1), real_val, null_val=0.0)
        #print("mae after calc_metrics",mae)
        #print("predict.squeeze shape",predict.squeeze(1).shape)
        #print("real val shape",real_val.shape)
        mae.backward()
        if self.clip is not None:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.clip)
        self.optimizer.step()
        #print("mae.item().shape",mae.item())
        return mae.item(),mape.item(),rmse.item()

    def eval(self, input, real_val):
        self.model.eval()
        input = nn.functional.pad(input,(1,0,0,0))
        output = self.model(input).transpose(1,3) #  [batch_size,seq_length,num_nodes,1]
       # print("outputshape in engine.eval",output.shape)
        real = torch.unsqueeze(real_val,dim=1)
        #print("real shape in engine.eval",real.shape)
        predict = self.scaler.inverse_transform(output)
        predict = torch.clamp(predict, min=0., max=70.)
        #print("predict shape in engine.eval",predict.shape)
        mae, mape, rmse = [x.item() for x in calc_metrics(predict, real, null_val=0.0)]
        return mae, mape, rmse

**EXP RESULTS**

In [ ]:
def summary(d):
    try:
        tr_val = pd.read_csv(f'{d}/metrics.csv', index_col=0)
        tr_ser = tr_val.loc[tr_val.valid_loss.idxmin()]
        tr_ser['best_epoch'] = tr_val.valid_loss.idxmin()
        tr_ser['min_train_loss'] = tr_val.train_loss.min()
    except FileNotFoundError:
        tr_ser = pd.Series()
    try:
        tmet = pd.read_csv(f'{d}/test_metrics.csv', index_col=0)
        tmean = tmet.add_prefix('test_').mean()

    except FileNotFoundError:
        tmean = pd.Series()
    tab = pd.concat([tr_ser, tmean]).round(3)
    return tab

def loss_curve(d):
    if 'logs' not in d: d =  f'logs/{d}'
    tr_val = pd.read_csv(f'{d}/metrics.csv', index_col=0)
    return tr_val[['train_loss', 'valid_loss']]


def plot_loss_curve(log_dir):
    d = loss_curve(log_dir)
    ax = d.plot()
    plt.axhline(d.valid_loss.min())
    print(d.valid_loss.idxmin())

def make_results_table():
    return pd.DataFrame({os.path.basename(c): summary(c) for c in glob('logs/*')}).T.sort_values('valid_loss')

**GRID SEARCH FOR HYPERPARAMETER TUNING**

In [ ]:
#Iterating over generated train&test datasets
import os
import itertools
directory_list = list()
for root, dirs, files in os.walk("/content/gdrive/MyDrive/Robbery_Daily_data", topdown=False):
    for name in dirs:
        directory_list.append(os.path.join(root, name))
import glob
paths=("/content/gdrive/MyDrive/MainFiles/CenterSide/AdjMatrix")
all_files = glob.glob(paths + "/*.pkl")
##Separate parameters
lr=[0.001,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1]
weight_decay=[0.0001,0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01]
lr_decay_rate=[0.97,0.94,0.9375,0.9350,0.9325,0.93,0.9275,0.9250,0.9225,0.92,0.9175]
for k,l,n in zip(lr,weight_decay,lr_decay_rate):
    hdir="/content/gdrive/MyDrive/"
    best_model_save_path = os.path.join(hdir, 'best_model.pth')
    def main(args, **model_kwargs):
        device = torch.device(args.device)
        data = load_dataset(args.data, args.batch_size, args.batch_size, args.batch_size, n_obs=args.n_obs, fill_zeroes=args.fill_zeroes)
        scaler = data['scaler']
        aptinit, supports = make_graph_inputs(args, device)
        torch.manual_seed(0)
        model = GWNet.from_args(args, device, supports, aptinit, **model_kwargs)
        if args.checkpoint:
          model.load_checkpoint(torch.load(args.checkpoint))
        model.to(device)
        engine = Trainer.from_args(model, scaler, args)
        metrics = []
        best_model_save_path = os.path.join(hdir, 'best_model.pth')
        lowest_mae_yet = 100  # high value, will get overwritten
        mb = progress_bar(list(range(1, args.epochs + 1)))
        epochs_since_best_mae = 0
        for _ in mb:
            train_loss, train_mape, train_rmse = [], [], []
            data['train_loader'].shuffle()
            for iter, (x, y) in enumerate(data['train_loader'].get_iterator()):
              trainx = torch.Tensor(x).to(device).transpose(1, 3)
              trainy = torch.Tensor(y).to(device).transpose(1, 3)
              yspeed = trainy[:, 0, :, :]
              if yspeed.max() == 0: continue
                  #print('traınx',trainx.shape)
                  #print('traıny',trainy.shape)
                  #print('yspeed',yspeed.shape)
              mae, mape, rmse = engine.train(trainx, yspeed)
              train_loss.append(mae)
              train_mape.append(mape)
              train_rmse.append(rmse)
              if args.n_iters is not None and iter >= args.n_iters:
                  break
            engine.scheduler.step()
            _, valid_loss, valid_mape, valid_rmse = eval_(data['val_loader'], device, engine)
            m = dict(train_loss=np.mean(train_loss), train_mape=np.mean(train_mape),
                      train_rmse=np.mean(train_rmse), valid_loss=np.mean(valid_loss),
                        valid_mape=np.mean(valid_mape), valid_rmse=np.mean(valid_rmse))

            m = pd.Series(m)
            metrics.append(m)
            if m.valid_loss < lowest_mae_yet:
                torch.save(engine.model.state_dict(), best_model_save_path)
                lowest_mae_yet = m.valid_loss
                epochs_since_best_mae = 0
            else:
                epochs_since_best_mae += 1
            met_df = pd.DataFrame(metrics)
            mb.comment = f'best val_loss: {met_df.valid_loss.min(): .3f}, current val_loss: {m.valid_loss:.3f}, current train loss: {m.train_loss: .3f}'
            met_df.round(6).to_csv(f'{hdir}/metrics.csv')
            if epochs_since_best_mae >= args.es_patience: break
            # Metrics on test data
        with open(best_model_save_path, 'rb') as f:
          buffer = io.BytesIO(f.read())
        engine.model.load_state_dict(torch.load(buffer))
        realy = torch.Tensor(data['y_test']).transpose(1, 3)[:, 0, :, :].to(device)
        test_met_df, yhat = calc_tstep_metrics(engine.model, device, data['test_loader'], scaler, realy, args.seq_length)
        test_met_df.round(6).to_csv(os.path.join(hdir, 'test_metrics.csv'))

    def eval_(ds, device, engine):
        """Run validation."""
        valid_loss = []
        valid_mape = []
        valid_rmse = []
        s1 = time.time()
        for (x, y) in ds.get_iterator():
            testx = torch.Tensor(x).to(device).transpose(1, 3)
            testy = torch.Tensor(y).to(device).transpose(1, 3)
            metrics = engine.eval(testx, testy[:, 0, :, :])
            valid_loss.append(metrics[0])
            valid_mape.append(metrics[1])
            valid_rmse.append(metrics[2])
        total_time = time.time() - s1
        return total_time, valid_loss, valid_mape, valid_rmse


    if __name__ == "__main__":
        parser =get_shared_arg_parser()
        parser.add_argument('--epochs', type=int, default=100, help='')
        parser.add_argument('--clip', type=int, default=3, help='Gradient Clipping')
        parser.add_argument('--weight_decay', type=float, default=l, help='weight decay rate')
        parser.add_argument('--learning_rate', type=float, default=k, help='learning rate')
        parser.add_argument('--lr_decay_rate', type=float, default=n, help='learning rate')
        parser.add_argument('--save', type=str, default='Robbery_Daily_experiment', help='save path')
        parser.add_argument('--n_iters', default=None, help='quit after this many iterations')
        parser.add_argument('--es_patience', type=int, default=25, help='quit if no improvement after this many iterations')

        args, unknown = parser.parse_known_args()
        t1 = time.time()
        if not os.path.exists(hdir):
            os.mkdir(hdir)
        pickle_save(args,'/content/gdrive/MyDrive/args.pkl')
        main(args)
        t2 = time.time()
        mins = (t2 - t1) / 60
        print(f"Total time spent: {mins:.2f} seconds")
        #pref=m.split('/')[-1]
        #pp=p.split('/')[-1].split('.')[0].split('graph')[0].split('Center')[1]
        key=str("experiment:"+"-"+ str(k)+"-"+str(l)+"-"+str(n)+".csv")
        epath=('/content/gdrive/MyDrive/Robbery_Daily_experiment/%s'%(key))
    def main(args, save_pred_path=epath, save_metrics_path='last_test_metrics_Robbery_preds_10day_dayca8_smoothingcoef05.csv', loader='test', **model_kwargs):
        args.checkpoint= best_model_save_path
        device = torch.device(args.device)
        adjinit, supports = make_graph_inputs(args, device)
        model = GWNet.from_args(args, device, supports, adjinit,**model_kwargs)
        model.load_state_dict(torch.load(args.checkpoint))
        model.to(device)
        model.eval()
        print('model loaded successfully')
        data = load_dataset(args.data, args.batch_size, args.batch_size, args.batch_size, n_obs=args.n_obs, fill_zeroes=args.fill_zeroes)
        scaler = data['scaler']
        realy = torch.Tensor(data[f'y_{loader}']).to(device)
        realy = realy.transpose(1,3)[:,0,:,:]
        met_df, yhat = calc_tstep_metrics(model, device, data[f'{loader}_loader'], scaler, realy, args.seq_length)
        df2 = make_pred_df(realy, yhat, scaler, args.seq_length)
        met_df.to_csv(save_metrics_path)
        df2.to_csv(save_pred_path, index=False)
        if args.plotheatmap: plot_learned_adj_matrix(model)
        return met_df, df2

    def plot_learned_adj_matrix(model):
        adp = F.softmax(F.relu(torch.mm(model.nodevec1, model.nodevec2)), dim=1)
        adp = adp.cpu().detach().numpy()
        adp = adp / np.max(adp)
        df = pd.DataFrame(adp)
        sns.heatmap(df, cmap="RdYlBu")
        plt.savefig("/content/gdrive/MyDrive/heatmap.png")


    if __name__ == "__main__":
        parser = get_shared_arg_parser()
        parser.add_argument('--checkpoints', type=str, help='')
        parser.add_argument('--plotheatmap', action='store_true')
        args, unknown = parser.parse_known_args()
        main(args)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:194: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


Total time spent: 33.92 seconds
model loaded successfully


Total time spent: 30.09 seconds
model loaded successfully


Total time spent: 22.93 seconds
model loaded successfully


Total time spent: 23.67 seconds
model loaded successfully


...

Total time spent: 29.18 seconds
model loaded successfully


Total time spent: 32.12 seconds
model loaded successfully


Total time spent: 31.99 seconds
model loaded successfully


Total time spent: 24.96 seconds
model loaded successfully


Total time spent: 29.93 seconds
model loaded successfully


# **GRID SEARCH TRAIN-TEST**

In [ ]:
#Iterating over generated train&test datasets
import os
directory_list = list()
for root, dirs, files in os.walk("/content/gdrive/MyDrive/Robbery_Daily_data", topdown=False):
    for name in dirs:
        directory_list.append(os.path.join(root, name))
import glob
paths=("/content/gdrive/MyDrive/MainFiles/CenterSide/AdjMatrix")
all_files = glob.glob(paths + "/*.pkl")

for m in directory_list:
  for p in all_files:
      import itertools
      hdir="/content/gdrive/MyDrive/"
      best_model_save_path = os.path.join(hdir, 'best_model.pth')
      def main(args, **model_kwargs):
          device = torch.device(args.device)
          data = load_dataset(args.data, args.batch_size, args.batch_size, args.batch_size, n_obs=args.n_obs, fill_zeroes=args.fill_zeroes)
          scaler = data['scaler']
          aptinit, supports = make_graph_inputs(args, device)
          torch.manual_seed(0)
          model = GWNet.from_args(args, device, supports, aptinit, **model_kwargs)
          if args.checkpoint:
            model.load_checkpoint(torch.load(args.checkpoint))
          model.to(device)
          engine = Trainer.from_args(model, scaler, args)
          metrics = []
          best_model_save_path = os.path.join(hdir, 'best_model.pth')
          lowest_mae_yet = 100  # high value, will get overwritten
          mb = progress_bar(list(range(1, args.epochs + 1)))
          epochs_since_best_mae = 0
          for _ in mb:
              train_loss, train_mape, train_rmse = [], [], []
              data['train_loader'].shuffle()
              for iter, (x, y) in enumerate(data['train_loader'].get_iterator()):
                trainx = torch.Tensor(x).to(device).transpose(1, 3)
                trainy = torch.Tensor(y).to(device).transpose(1, 3)
                yspeed = trainy[:, 0, :, :]
                if yspeed.max() == 0: continue
                    #print('traınx',trainx.shape)
                    #print('traıny',trainy.shape)
                    #print('yspeed',yspeed.shape)
                mae, mape, rmse = engine.train(trainx, yspeed)
                train_loss.append(mae)
                train_mape.append(mape)
                train_rmse.append(rmse)
                if args.n_iters is not None and iter >= args.n_iters:
                    break
              engine.scheduler.step()
              _, valid_loss, valid_mape, valid_rmse = eval_(data['val_loader'], device, engine)
              m = dict(train_loss=np.mean(train_loss), train_mape=np.mean(train_mape),
                        train_rmse=np.mean(train_rmse), valid_loss=np.mean(valid_loss),
                          valid_mape=np.mean(valid_mape), valid_rmse=np.mean(valid_rmse))

              m = pd.Series(m)
              metrics.append(m)
              if m.valid_loss < lowest_mae_yet:
                  torch.save(engine.model.state_dict(), best_model_save_path)
                  lowest_mae_yet = m.valid_loss
                  epochs_since_best_mae = 0
              else:
                  epochs_since_best_mae += 1
              met_df = pd.DataFrame(metrics)
              mb.comment = f'best val_loss: {met_df.valid_loss.min(): .3f}, current val_loss: {m.valid_loss:.3f}, current train loss: {m.train_loss: .3f}'
              met_df.round(6).to_csv(f'{hdir}/metrics.csv')
              if epochs_since_best_mae >= args.es_patience: break
              # Metrics on test data
          with open(best_model_save_path, 'rb') as f:
            buffer = io.BytesIO(f.read())
          engine.model.load_state_dict(torch.load(buffer))
          realy = torch.Tensor(data['y_test']).transpose(1, 3)[:, 0, :, :].to(device)
          test_met_df, yhat = calc_tstep_metrics(engine.model, device, data['test_loader'], scaler, realy, args.seq_length)
          test_met_df.round(6).to_csv(os.path.join(hdir, 'test_metrics.csv'))

      def eval_(ds, device, engine):
          """Run validation."""
          valid_loss = []
          valid_mape = []
          valid_rmse = []
          s1 = time.time()
          for (x, y) in ds.get_iterator():
              testx = torch.Tensor(x).to(device).transpose(1, 3)
              testy = torch.Tensor(y).to(device).transpose(1, 3)
              metrics = engine.eval(testx, testy[:, 0, :, :])
              valid_loss.append(metrics[0])
              valid_mape.append(metrics[1])
              valid_rmse.append(metrics[2])
          total_time = time.time() - s1
          return total_time, valid_loss, valid_mape, valid_rmse


      if __name__ == "__main__":
          parser =get_shared_arg_parser()
          parser.add_argument('--epochs', type=int, default=100, help='')
          parser.add_argument('--clip', type=int, default=3, help='Gradient Clipping')
          parser.add_argument('--weight_decay', type=float, default=0.001, help='weight decay rate')
          parser.add_argument('--learning_rate', type=float, default=0.01, help='learning rate')
          parser.add_argument('--lr_decay_rate', type=float, default=0.97, help='learning rate')
          parser.add_argument('--save', type=str, default='experiment', help='save path')
          parser.add_argument('--n_iters', default=None, help='quit after this many iterations')
          parser.add_argument('--es_patience', type=int, default=25, help='quit if no improvement after this many iterations')

          args, unknown = parser.parse_known_args()
          t1 = time.time()
          if not os.path.exists(hdir):
              os.mkdir(hdir)
          pickle_save(args,'/content/gdrive/MyDrive/args.pkl')
          main(args)
          t2 = time.time()
          mins = (t2 - t1) / 60
          print(f"Total time spent: {mins:.2f} seconds")
          pref=m.split('/')[-1]
          pp=p.split('/')[-1].split('.')[0].split('graph')[0].split('Center')[1]
          key=str("experiment:"+pref+pp+"-"+".csv")
          epath=('/content/gdrive/MyDrive/Robbery_Daily_experiment/%s'%(key))
      def main(args, save_pred_path=epath, save_metrics_path='last_test_metrics_Robbery_preds_10day_dayca8_smoothingcoef05.csv', loader='test', **model_kwargs):
          args.checkpoint= best_model_save_path
          device = torch.device(args.device)
          adjinit, supports = make_graph_inputs(args, device)
          model = GWNet.from_args(args, device, supports, adjinit,**model_kwargs)
          model.load_state_dict(torch.load(args.checkpoint))
          model.to(device)
          model.eval()
          print('model loaded successfully')
          data = load_dataset(args.data, args.batch_size, args.batch_size, args.batch_size, n_obs=args.n_obs, fill_zeroes=args.fill_zeroes)
          scaler = data['scaler']
          realy = torch.Tensor(data[f'y_{loader}']).to(device)
          realy = realy.transpose(1,3)[:,0,:,:]
          met_df, yhat = calc_tstep_metrics(model, device, data[f'{loader}_loader'], scaler, realy, args.seq_length)
          df2 = make_pred_df(realy, yhat, scaler, args.seq_length)
          met_df.to_csv(save_metrics_path)
          df2.to_csv(save_pred_path, index=False)
          if args.plotheatmap: plot_learned_adj_matrix(model)
          return met_df, df2

      def plot_learned_adj_matrix(model):
          adp = F.softmax(F.relu(torch.mm(model.nodevec1, model.nodevec2)), dim=1)
          adp = adp.cpu().detach().numpy()
          adp = adp / np.max(adp)
          df = pd.DataFrame(adp)
          sns.heatmap(df, cmap="RdYlBu")
          plt.savefig("/content/gdrive/MyDrive/heatmap.png")


      if __name__ == "__main__":
          parser = get_shared_arg_parser()
          parser.add_argument('--checkpoints', type=str, help='')
          parser.add_argument('--plotheatmap', action='store_true')
          args, unknown = parser.parse_known_args()
          main(args)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:194: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


Total time spent: 19.52 seconds
model loaded successfully


Total time spent: 19.13 seconds
model loaded successfully


Total time spent: 19.16 seconds
model loaded successfully


Total time spent: 19.24 seconds
model loaded successfully


...

Total time spent: 19.14 seconds
model loaded successfully


Total time spent: 19.11 seconds
model loaded successfully


Total time spent: 19.20 seconds
model loaded successfully


Total time spent: 19.08 seconds
model loaded successfully


Total time spent: 19.11 seconds
model loaded successfully


**PREPARING THE EXTERNAL DATASETS**

In [ ]:
import geopandas as gpd
cl_gchi_core_sn=pd.read_pickle("/content/gdrive/MyDrive/MainFiles/snappedcorecrimes.pkl")
##Subsetting the crime dataset
Robbery=['312','313','031A','031B','320','325','326','330','331','334','337','033A','033B','340']
Theft=['810','820','850','860','865','870','880','890','895']
core20162018=cl_gchi_core_sn.loc[(cl_gchi_core_sn['Time_ID'] > 2191) & (cl_gchi_core_sn['Time_ID'] <= 4383)]
Robbery20162018=core20162018.loc[(core20162018.IUCR.isin(Robbery))]
Theft20162018=core20162018.loc[(core20162018.IUCR.isin(Theft))]
##Retrieving the segments
gchi_segments=pd.read_pickle("/content/gdrive/MyDrive/MainFiles/segments.pkl")
gchi_intersections=pd.read_pickle("/content/gdrive/MyDrive/MainFiles/gintersections.pkl")

**RETRIEVING THE BUSINESS**

In [ ]:
##Retrieving the business
chi_bus= pd.read_csv("/content/gdrive/MyDrive/MainFiles/MatchedFeatures8.csv")
##Theft RSIS
tchi_bus=chi_bus.drop(['Theft_RSIS_WD', 'Theft_RSIS_WE','Robbery_RSIS_WD', 'Robbery_RSIS_WE',
                       'Theft_RSIS_FS','Theft_RSIS_SS','Theft_RSIS_TS' ], axis=1)
tchi_bus.head()

FileNotFoundError: ignored

**Generating RSIS datasets**

In [ ]:
def generate_datasets(path,crime_type,TransactionDF,matrixdf,day=True,shift=False):
    df=pd.read_csv(path)
    if day:
        names=['Segment_ID',str(crime_type+'_RSIS_WD'),str(crime_type+'_RSIS_WE')]
        tchi_bus=df[names]
        a=tchi_bus.groupby('Segment_ID')
        bus_Segment=a.sum().transpose()
        bus_Segment.columns = bus_Segment.columns.astype(str)
        bus_Segment.index=[0,1]
        wdf=pd.DataFrame(columns=['st1','st2','influence'])
        wef=pd.DataFrame(columns=['st1','st2','influence'])
    if shift:
        names=['Segment_ID',str(crime_type+'_RSIS_FS'),str(crime_type+'_RSIS_SS'),str(crime_type+'_RSIS_TS')]
        tchi_bus=df[names]
        a=tchi_bus.groupby('Segment_ID')
        bus_Segment=a.sum().transpose()
        bus_Segment.columns = bus_Segment.columns.astype(str)
        bus_Segment.index=[0,1,2]
        wfs=pd.DataFrame(columns=['st1','st2','influence'])
        wss=pd.DataFrame(columns=['st1','st2','influence'])
        wts=pd.DataFrame(columns=['st1','st2','influence'])

    for keys,values in matrixdf.items():
        if keys in bus_Segment.columns:
            for key,value in values.items():
                for k,v in value.items():
                    if day:
                        wdi=bus_Segment.loc[0,keys]*v
                        wei=bus_Segment.loc[1,keys]*v
                        wddct={'st1':keys,'st2':key,'influence':wdi}
                        wedct={'st1':keys,'st2':key,'influence':wei}
                        wdf=wdf.append(wddct,ignore_index=True)
                        wef=wef.append(wedct,ignore_index=True)
                    if shift:
                        wfsi=bus_Segment.loc[0,keys]*v
                        wssi=bus_Segment.loc[1,keys]*v
                        wtsi=bus_Segment.loc[2,keys]*v
                        wfsdct={'st1':keys,'st2':key,'influence':wfsi}
                        wssdct={'st1':keys,'st2':key,'influence':wssi}
                        wtsdct={'st1':keys,'st2':key,'influence':wtsi}
                        wfs=wfs.append(wfsdct,ignore_index=True)
                        wss=wss.append(wssdct,ignore_index=True)
                        wts=wts.append(wtsdct,ignore_index=True)
    if day:
        df_list=[wdf,wef]
        lst1=[]
        for i in df_list:
            b=i.groupby('st2')
            c=b.sum().transpose()
            lst1.append(c)
        df1=pd.DataFrame(columns=TransactionDF.columns)
        b=np.zeros(2288)
        df1=df1.append(pd.DataFrame(b.reshape(2,-1), columns=TransactionDF.columns), ignore_index=True)
        df2 = pd.concat(lst1)
        df2.columns = df2.columns.astype(str)
        df2.index=[0,1]
        for index, row in df2.iterrows():
            for i in list(df2.columns):
                df1.loc[index,i]=df2.loc[index,i]
        df3=pd.DataFrame(columns=TransactionDF.columns)
        b=np.zeros(2288)
        df3=df3.append(pd.DataFrame(b.reshape(2,-1), columns=TransactionDF.columns), ignore_index=True)
    elif shift:
        df_list=[wfs,wss,wts]
        lst1=[]
        for i in df_list:
            b=i.groupby('st2')
            c=b.sum().transpose()
            lst1.append(c)
        df1=pd.DataFrame(columns=TransactionDF.columns)
        b=np.zeros(3432)
        df1=df1.append(pd.DataFrame(b.reshape(3,-1), columns=TransactionDF.columns), ignore_index=True)
        df2 = pd.concat(lst1)
        df2.columns = df2.columns.astype(str)
        df2.index=[0,1,2]
        for index, row in df2.iterrows():
            for i in list(df2.columns):
                df1.loc[index,i]=df2.loc[index,i]
        df3=pd.DataFrame(columns=TransactionDF.columns)
        b=np.zeros(3432)
        df3=df3.append(pd.DataFrame(b.reshape(3,-1), columns=TransactionDF.columns), ignore_index=True)
    for index, row in bus_Segment.iterrows():
            for i in list(bus_Segment.columns):
                df3.loc[index,i]=bus_Segment.loc[index,i]
    df4= df1.add(df3, fill_value=0)
    #Normalize the dataframe
    column_maxes = df4.max()
    df_max = column_maxes.max()
    normalized_df4 = df4 / df_max
    if day:
        a= pd.date_range(start ='1-1-2016',end ='31-12-2017', freq ='D')
        df5=pd.DataFrame(columns=TransactionDF.columns)
        df6=pd.DataFrame(columns=TransactionDF.columns)
#Weekend/Weekday distinction
        for i in a:
            if i.dayofweek>4:
                df5=df5.append(pd.Series(df4.values[1],index=df5.columns),ignore_index=True)
                df6=df6.append(pd.Series(normalized_df4.values[1],index=df6.columns),ignore_index=True)
            else:
                df5=df5.append(pd.Series(df4.values[1],index=df5.columns),ignore_index=True)
                df6=df6.append(pd.Series(normalized_df4.values[0],index=df6.columns),ignore_index=True)
    if shift:
        a= pd.date_range(start ='1-1-2016 00:00:00',end ='31-12-2017 08:00:00', freq ='8H')
        df5=pd.DataFrame(columns=TransactionDF.columns)
        df6=pd.DataFrame(columns=TransactionDF.columns)
        for i in a:
            if i.hour==0:
                df5=df5.append(pd.Series(normalized_df4.values[0],index=df5.columns),ignore_index=True)
                df6=df6.append(pd.Series(normalized_df4.values[0],index=df6.columns),ignore_index=True)
            elif i.hour==8:
                df5=df5.append(pd.Series(normalized_df4.values[1],index=df5.columns),ignore_index=True)
                df6=df6.append(pd.Series(normalized_df4.values[1],index=df6.columns),ignore_index=True)
            else:
                df5=df5.append(pd.Series(normalized_df4.values[2],index=df5.columns),ignore_index=True)
                df6=df6.append(pd.Series(normalized_df4.values[2],index=df6.columns),ignore_index=True)
    return df5,df6

**Writing the files to csv**

In [ ]:
crime_list=['Robbery','Theft']
order=[[True,False],[False,True]]
path='C:/Users/Tugrul/Desktop/MatchedFeatures8.csv'
for i in crime_list:
    for j in order:
        if j[0]:
            a,b=generate_datasets(path,i,Theft_Transactions8_Shift,matrix8,day=j[0],shift=j[1])
            name1=str(i+'day.csv')
            name2=str(i+'normalized_day.csv')
            a.to_csv('/content/gdrive/MyDrive/MainFiles/RSIS/ %s' % (name1))
            b.to_csv('/content/gdrive/MyDrive/MainFiles/RSIS/ %s' % (name2))
        if j[1]:
            a,b=generate_datasets(path,i,Theft_Transactions8_Shift,matrix8,day=j[0],shift=j[1])
            name3=str(i+'shift.csv')
            name4=str(i+'normalized_shift.csv')
            a.to_csv('/content/gdrive/MyDrive/MainFiles/RSIS/ %s' % (name3))
            b.to_csv('/content/gdrive/MyDrive/MainFiles/RSIS/ %s' % (name4))

**Writing the RSIS counts**

In [ ]:
crime_list=['Robbery','Theft']
order=[[True,False],[False,True]]
path='/content/gdrive/MyDrive/MainFiles/CenterSide/RSIS/matched_features_Center.csv'
for i in crime_list:
    for j in order:
        if j[0]:
            a,b=generate_datasets_counts(path,i,TheftTable_daily,day=j[0],shift=j[1])
            name1=str(i+'count_day.csv')
            name2=str(i+'count_normalized_day.csv')
            a.to_csv('/content/gdrive/MyDrive/MainFiles/CenterSide/RSIS/ %s' % (name1))
            b.to_csv('/content/gdrive/MyDrive/MainFiles/CenterSide/RSIS/ %s' % (name2))
        if j[1]:
            a,b=generate_datasets_counts(path,i,TheftTable,day=j[0],shift=j[1])
            name3=str(i+'count_shift.csv')
            name4=str(i+'count_normalized_shift.csv')
            a.to_csv('/content/gdrive/MyDrive/MainFiles/CenterSide/RSIS/ %s' % (name3))
            b.to_csv('/content/gdrive/MyDrive/MainFiles/CenterSide/RSIS/ %s' % (name4))

NameError: ignored

**Retrieving RSIS**

In [ ]:
##Theft
#Shift
TheftRSIS = pd.read_csv('/content/gdrive/MyDrive/MainFiles/CenterSide/RSIS/ Theftshift.csv')
TheftRSIS=TheftRSIS.drop(columns=['Unnamed: 0'])
TheftRSIS_normalized = pd.read_csv('/content/gdrive/MyDrive/MainFiles/CenterSide/RSIS/ Theftnormalized_shift.csv')
TheftRSIS_normalized=TheftRSIS_normalized.drop(columns=['Unnamed: 0'])
TheftRSIS_mult_normalized = pd.read_csv('/content/gdrive/MyDrive/MainFiles/CenterSide/RSIS/ Theftmult_normalized_shift.csv')
TheftRSIS_mult_normalized=TheftRSIS_mult_normalized.drop(columns=['Unnamed: 0'])
TheftRSIS_count_normalized = pd.read_csv('/content/gdrive/MyDrive/MainFiles/CenterSide/RSIS/ Theftcount_normalized_shift.csv')
TheftRSIS_count_normalized=TheftRSIS_count_normalized.drop(columns=['Unnamed: 0'])
#Daily
TheftRSIS_daily = pd.read_csv('/content/gdrive/MyDrive/MainFiles/CenterSide/RSIS/ Theftday.csv')
TheftRSIS_daily=TheftRSIS_daily.drop(columns=['Unnamed: 0'])
TheftRSIS_normalized_daily = pd.read_csv('/content/gdrive/MyDrive/MainFiles/CenterSide/RSIS/ Theftnormalized_day.csv')
TheftRSIS_normalized_daily=TheftRSIS_normalized_daily.drop(columns=['Unnamed: 0'])
TheftRSIS_mult_normalized_daily = pd.read_csv('/content/gdrive/MyDrive/MainFiles/CenterSide/RSIS/ Theftmult_normalized_day.csv')
TheftRSIS_mult_normalized_daily=TheftRSIS_mult_normalized_daily.drop(columns=['Unnamed: 0'])
TheftRSIS_count_normalized_daily = pd.read_csv('/content/gdrive/MyDrive/MainFiles/CenterSide/RSIS/ Theftcount_normalized_day.csv')
TheftRSIS_count_normalized_daily=TheftRSIS_count_normalized_daily.drop(columns=['Unnamed: 0'])
##Robbery
#Shift
RobberyRSIS = pd.read_csv('/content/gdrive/MyDrive/MainFiles/CenterSide/RSIS/ Robberyshift.csv')
RobberyRSIS=RobberyRSIS.drop(columns=['Unnamed: 0'])
RobberyRSIS_normalized = pd.read_csv('/content/gdrive/MyDrive/MainFiles/CenterSide/RSIS/ Robberynormalized_shift.csv')
RobberyRSISS_normalized=RobberyRSIS_normalized.drop(columns=['Unnamed: 0'])
RobberyRSIS_mult_normalized = pd.read_csv('/content/gdrive/MyDrive/MainFiles/CenterSide/RSIS/ Robberymult_normalized_shift.csv')
RobberyRSIS_mult_normalized=RobberyRSIS_mult_normalized.drop(columns=['Unnamed: 0'])
RobberyRSIS_count_normalized = pd.read_csv('/content/gdrive/MyDrive/MainFiles/CenterSide/RSIS/ Robberycount_normalized_shift.csv')
RobberyRSIS_count_normalized=RobberyRSIS_count_normalized.drop(columns=['Unnamed: 0'])
#Daily
RobberyRSIS_daily = pd.read_csv('/content/gdrive/MyDrive/MainFiles/CenterSide/RSIS/ Robberyday.csv')
RobberyRSIS_daily=RobberyRSIS_daily.drop(columns=['Unnamed: 0'])
RobberyRSIS_normalized_daily = pd.read_csv('/content/gdrive/MyDrive/MainFiles/CenterSide/RSIS/ Robberynormalized_day.csv')
RobberyRSIS_normalized_daily=RobberyRSIS_normalized_daily.drop(columns=['Unnamed: 0'])
RobberyRSIS_mult_normalized_daily = pd.read_csv('/content/gdrive/MyDrive/MainFiles/CenterSide/RSIS/ Robberymult_normalized_day.csv')
RobberyRSIS_mult_normalized_daily=RobberyRSIS_mult_normalized_daily.drop(columns=['Unnamed: 0'])
RobberyRSIS_count_normalized_daily = pd.read_csv('/content/gdrive/MyDrive/MainFiles/CenterSide/RSIS/ Robberycount_normalized_day.csv')
RobberyRSIS_count_normalized_daily = RobberyRSIS_count_normalized_daily.iloc[: , 1:]
RobberyRSIS_count_normalized_daily.head()

,1,3,5,6,7,8,9,10,11,19,20,28,30,41,55,71,99,103,111,121,125,129,135,152,173,180,187,189,195,202,213,243,318,330,348,357,371,381,402,408,...,55344,55356,55359,55421,55426,55433,55439,55469,55490,55506,55520,55550,55573,55588,55592,55597,55636,55687,55689,55752,55772,55780,55814,55831,55840,55867,55915,55932,55990,56015,56025,56039,56141,56145,56146,56156,56183,56225,56250,56293
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.2,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.2,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.2,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.2,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.2,0.0


**SUBSETTING THE GRAPH**

In [ ]:
def create_neighborhood_graph(Cno):
    a=gchi_segments['Community_Area']==Cno
    #segmentsCno=gchi_segments[gchi_segments.Segment_ID.isin(Cno)]
    segmentsCno=gchi_segments[a]
    edgesCno=segmentsCno[['i', 'j','Segment_ID','shape_len']]
    edgesCno['i']=edgesCno['i'].astype('str')
    edgesCno['j']=edgesCno['j'].astype('str')
    cgraphCno=nx.Graph()
    for i, elrow in edgesCno.iterrows():## iterrows help iterate over DataFrame rows as (index, Series) pairs
        cgraphCno.add_edge(elrow[0], elrow[1], attr_dict=elrow[2:].to_dict())
    ##Intersections
    un1=np.unique(edgesCno['i'])
    un2=np.unique(edgesCno['j'])
    un3=np.unique(np.concatenate((un1,un2)))
    nodesCno=gchi_intersections[gchi_intersections['num'].isin(un3)]
    nodesCno['num']=nodesCno['num'].astype('str')
    for i, nlrow in nodesCno.iterrows():
        cgraphCno.nodes[nlrow['num']].update(nlrow[1:].to_dict())
    ##Listing the first 5 edges and nodes
    #list(cgraphCno.edges(data=True))[0:5]
    #list(cgraphCno.nodes(data=True))[0:5]
    ##Finding the number of components
    #nx.number_connected_components(cgraphCno)
    ##CREATING THE SEGMENT GRAPH FOR COMMUNITY AREA 8
    mnsegmentsCno=segmentsCno[['Segment_ID','shape_len','i','j','Community_Area']]
    ##Converting Types
    mnsegmentsCno['Community_Area']=mnsegmentsCno['Community_Area'].astype('Int64')
    mnsegmentsCno['Segment_ID']=mnsegmentsCno['Segment_ID'].astype('str')
    mnsegmentsCno['i']=mnsegmentsCno['i'].astype('str')
    mnsegmentsCno['j']=mnsegmentsCno['j'].astype('str')
    mnsegmentsCno['Label'] = mnsegmentsCno.apply(lambda row: str(str(row.i)+'-'+str(row.j)), axis = 1)
    ##Adjacency matrix of maingraph
    #neighborsCno= nx.adjacency_matrix(cgraphCno, nodelist=None, weight='weight')
    ##Neighbor List
    matrixxCno= nx.convert.to_dict_of_dicts(cgraphCno)
    ##Find the network distance between neighbors
    medge_dict={}
    for u,v,data in cgraphCno.edges(data=True):
       key= str(str(u)+'-'+str(v))
       medge_dict.update({key:data})
    ##Converting Segments into Nodes
    import itertools
    caCno= pd.DataFrame()
    for key, value in matrixxCno.items():
        keylist=value.keys()
        combinations_object = itertools.combinations(keylist, 2)
        combinations_list = list(combinations_object)
        for i in combinations_list:
            nkey1= str(key+'-'+i[0])
            rnkey1=str(i[0]+'-'+key)
            c1= mnsegmentsCno.loc[(mnsegmentsCno['Label'] == nkey1) | (mnsegmentsCno['Label'] == rnkey1) ,'Segment_ID'].iloc[0]
            d1= (mnsegmentsCno.loc[(mnsegmentsCno['Label'] == nkey1) | (mnsegmentsCno['Label'] == rnkey1),'shape_len'].iloc[0])/2
            nkey2= str(key+'-'+i[1])
            rnkey2=rnkey1=str(i[1]+'-'+key)
            c2= mnsegmentsCno.loc[(mnsegmentsCno['Label'] == nkey2) | (mnsegmentsCno['Label'] == rnkey2),'Segment_ID'].iloc[0]
            d2= (mnsegmentsCno.loc[(mnsegmentsCno['Label'] == nkey2) | (mnsegmentsCno['Label'] == rnkey2),'shape_len'].iloc[0])/2
            e=(d1+d2)
            caCno = caCno.append({'S1': c1, 'S2': c2, 'distance': e}, ignore_index=True)
    ##Weigthing the edges using gaussian weighting
    import math
    caCno['weight'] = caCno.apply(lambda row: (math.exp(-row.distance**0.2)), axis = 1)
    ##Creating three columns dataset
    #ncaCno=caCno[[['S1','S2','weight']]
    #ncaCno=ncaCno[.rename(columns={"S1": "from", "S2": "to","weight": "distance"})
    #ncaCno.to_csv('C:/Users/Tugrul/Desktop/distances.csv',sep=",",index=False)
    ##Creating Segment graph from the dataframe
    graphcaCno=nx.from_pandas_edgelist(caCno,source='S1',target='S2',edge_attr='weight')
    #ngraphca25=nx.from_pandas_edgelist(ca25,source='S1',target='S2',edge_attr='distance')
    ##creating dict of dicts for the dataframe
    #matrixca25= nx.convert.to_dict_of_dicts(graphca25)
    k='8'
    name= "graph.pickle"
    ##Saving the graph for CA 8
    with open('/content/gdrive/MyDrive/MainFiles/%s%s' % (k,name), 'wb') as handle:
        pickle.dump(graphcaCno, handle, protocol=2)
    return caCno,graphcaCno
gchi_segments=pd.read_pickle('/content/gdrive/MyDrive/MainFiles/segments.pkl')
gchi_intersections=pd.read_pickle('/content/gdrive/MyDrive/MainFiles/gintersections.pkl')
graphdf8,graph8=create_neighborhood_graph(8)
##Saving Graphdf8
graphdf8.to_pickle('/content/gdrive/MyDrive/MainFiles/graphdf8.pickle')
matrixca8= nx.convert.to_dict_of_dicts(graph8)

**CREATING TRANSACTION TABLE FOR COMMUNITY AREAS**

In [ ]:
def crime_transactions(sdf,ca):
    a=np.array(sdf['S1'],dtype=int)
    b=np.array(sdf['S2'],dtype=int)
    c=np.append(a,b)
    sdf=np.unique(c)
    nsdf= np.array2string(sdf)
    #np.savetxt('C:/Users/Tugrul/Desktop/graph_sensor_ids.txt', sdf, delimiter=',',fmt='%i')
    Columnnames= np.concatenate([['Time_ID'],sdf])
    Indexvalues=np.arange(2192,4384,1)
    a = np.zeros(shape=(2192,(len(sdf)+1)))
    sdfTransactionTable = pd.DataFrame(a)
    sdfTransactionTable.columns=Columnnames
    sdfTransactionTable['Time_ID']=Indexvalues
    sdfTransactionTable=sdfTransactionTable.astype(int)
    #Subsetting Robbery and Theft
    a=Robbery20162018['Community_Area']==ca
    Robberyca=Robbery20162018[a]
    a=Theft20162018['Community_Area']==ca
    Theftca=Theft20162018[a]
    #Robberyca=Robbery20162018[Robbery20162018.Segment_ID.isin(lst)]
    #Theftca=Theft20162018[Theft20162018.Segment_ID.isin(lst)]
    ## Creating Crosstabs for Selected Dataset
    selectedcrimes20162018={'Robbery':Robberyca,'Theft':Theftca}
    crimetimeseries={}
    for k,v in selectedcrimes20162018.items():
        df= pd.crosstab(v['Time_ID'],v['Segment_ID'],dropna=False)
        crimetimeseries.update({k:df})
    ##Inserting into the Robbery Transactional Table
    RobberySerie=crimetimeseries['Robbery']
    RobberySerie.dtypes
    RobberySerie.columns = RobberySerie.columns.astype(str)
    sdfTransactionTableR=sdfTransactionTable.copy()
    sdfTransactionTableR.index=sdfTransactionTableR['Time_ID']
    sdfTransactionTableR.dtypes
    sdfTransactionTableR=sdfTransactionTableR.astype(str)
    #type(list(sdfTransactionTableR.columns)[1])==list(sdfTransactionTableR)[1]
    sdfTransactionTableR.columns = sdfTransactionTableR.columns.astype(str)
    RobberySerie.index = RobberySerie.index.map(str)
    sdfTransactionTableR.index = sdfTransactionTableR.index.map(str)
    ##Inserting the transactions
    for index, row in RobberySerie.iterrows():
        for i in list(RobberySerie.columns):
            sdfTransactionTableR.loc[index,i]=RobberySerie.loc[index,i]
    #Aggregating Robbery at daily level
    a= pd.date_range(start ='1-1-2016 00:00:00',end ='31-12-2017 08:00:00', freq ='8H')
    sdfTransactionTableR.index=a
    sdfTransactionTableR=  sdfTransactionTableR.apply(pd.to_numeric)
    sdfTransactionTableR_daily = sdfTransactionTableR.resample('D').sum()
    ##Inserting into the Theft Transactional Table
    TheftSerie=crimetimeseries['Theft']
    TheftSerie.columns = TheftSerie.columns.astype(str)
    sdfTransactionTableT=sdfTransactionTable.copy()
    sdfTransactionTableT.index=sdfTransactionTableT['Time_ID']
    sdfTransactionTableT.dtypes
    sdfTransactionTableT=sdfTransactionTableT.astype(str)
    sdfTransactionTableT.columns = sdfTransactionTableT.columns.astype(str)
    TheftSerie.index = TheftSerie.index.map(str)
    sdfTransactionTableT.index = sdfTransactionTableT.index.map(str)
    ##Inserting the theft transactions
    for index, row in TheftSerie.iterrows():
        for i in list(TheftSerie.columns):
            sdfTransactionTableT.loc[index,i]=TheftSerie.loc[index,i]
    sdfTransactionTableT=sdfTransactionTableT.drop(columns= ['Time_ID'])
    ##Aggregating at daily level
    a= pd.date_range(start ='1-1-2016 00:00:00',end ='31-12-2017 08:00:00', freq ='8H')
    sdfTransactionTableT.index=a
    sdfTransactionTableT=  sdfTransactionTableT.apply(pd.to_numeric)
    sdfTransactionTableT_daily = sdfTransactionTableT.resample('D').sum()
    ##Smoothing the columns
    sdfTransactionTableR_Smoothed=sdfTransactionTableR.ewm(alpha=0.1).mean()
    sdfTransactionTableT_Smoothed=sdfTransactionTableT.ewm(alpha=0.1).mean()
    #return sdfTransactionTableT_daily,sdfTransactionTableT
    return sdfTransactionTableT,sdfTransactionTableT_daily
#Theft_Transactions8_Daily,Theft_Transactions8_Shift=crime_transactions(graphdf8,8)

**RESTRUCTURING THE NEIGHBORHOOD TRANSACTIONS**

In [ ]:
def neighborhood_transactions(path,sdf,matrixdf):
    df= pd.read_csv(path)
    df=df.rename(columns={"y_last": "Actual", "yhat_last": "Predicted"})
    a=np.array(sdf['S1'],dtype=int)
    b=np.array(sdf['S2'],dtype=int)
    nodelist=np.unique(np.append(a,b))
    rep_num = int((len(df)/len(nodelist)))
    nodes=np.tile(nodelist,rep_num)
    df['Segment']=nodes
    dict1={}
    for key,value in matrixdf.items():
        nlist=[]
        nlist.append(key)
        for keys,values in value.items():
            nlist.append(keys)
        a=str(key)+'neighbor'
        dict1[a]=nlist
    dict2={}
    df['Segment']=df['Segment'].astype(str)
    for key,value in dict1.items():
        a=key
        dict2[a]=df[df.Segment.isin(value)]
    dict3={}
    ind_value=np.arange(1,(rep_num+1),1)
    for key,value in dict2.items():
        value['Segment'] = pd.to_numeric(value['Segment'])
        a=list(sorted(np.unique(value.Segment)))
        append_act = '_actual'
        append_pre='_predicted'
        b1 = [str(sub) + append_act for sub in a]
        b2 = [str(sub) + append_pre for sub in a]
        b3 = [None]*(len(b1)+len(b2))
        b3[::2] = b1
        b3[1::2] = b2
        ndf=value[['Actual','Predicted']]
        e=ndf.values.reshape(rep_num,-1)
        nndf=pd.DataFrame(data=e,index=ind_value,columns=b3)
        d=key
        dict3[d]=nndf
    return dict3

**CALCULATING THE MEAN HIT RATE**

In [ ]:
from functools import partial, reduce
from statistics import mean
import math
def calc_MHR(transactions_dict,transaction_df,segments_df):
    ##transaction_dict is the output of neighborhood transaction,
    ##transaction df is the output of crime_transactions function
    ##segments_df is the output of create_neighborhood_graph function
    my_reduce = partial(pd.merge, left_index=True, right_index=True)
    a=reduce(my_reduce, transactions_dict.values())
    d=a.filter(regex=("_predicted_x"))
    d.columns = d.columns.str.rstrip('_predicted_x')
    d=d.iloc[:,~d.columns.duplicated()]
    d.columns=d.columns.astype(int)
    e = [int(numeric_string) for numeric_string in np.array(d.columns)]
    f=sorted(e)
    g=d[f]
    actual_values=transaction_df.iloc[-(len(g)):,]
    un1=np.unique(segments_df['S1'])
    un2=np.unique(segments_df['S2'])
    un3=np.unique(np.concatenate((un1,un2)))
    gchi_segments['Segment_ID']=gchi_segments['Segment_ID'].astype(str)
    un4= gchi_segments[gchi_segments.Segment_ID.isin(un3)]
    slengths=un4[['Segment_ID','shape_len']]
    pred_dict={}
    for (index1, row1),(index2,row2) in zip(g.iterrows(),actual_values.iterrows()):
        d=np.array(row1)
        e=np.array(row2)
        key=str(str(index1)+'.Day')
        nname=str(str(index1)+'slengths')
        nname=slengths.copy()
        nname['prediction']=d
        nname['actual']=e
        pred_dict[index1]=nname
    hrs=0
    hr_dict={}
    hr_list=[]
    for key,value in pred_dict.items():
        value['Risk_Rank'] = value['prediction'].rank(ascending = 0)
        #value['Risk_Rank'] = pd.to_numeric(value['Risk_Rank'])
        value = value.set_index('Risk_Rank')
        value = value.sort_index()
        pred_segs=[]
        a=0
        for index, row in value.iterrows():
            a+=row.shape_len
            if a>(0.20*value['shape_len'].sum()):
                break
            else:
               pred_segs.append(row.Segment_ID)
        selected_segments=value[value.Segment_ID.isin(pred_segs)]
        #hitsegments=len(selected_segments[(selected_segments['actual']!=0)])
        #allcrsegments=(len(value[(value['actual']!=0)]))
        hitsegments=selected_segments['actual'].sum()
        allcrsegments=value['actual'].sum()
        hr=hitsegments/allcrsegments
        hr_list.append(hr)
        keys=str(str(key)+'.HR')
        hr_dict[keys]=hr
        hrs+=hr
    nhrlist = [x for x in hr_list if math.isnan(x) == False]
    mhr=mean(nhrlist)
    return hr_dict,mhr,pred_dict

**WRITING THE RESULTS TO A DICTIONARY**

In [ ]:
graph8=pd.read_pickle('/content/gdrive/MyDrive/MainFiles/CenterSide/Graphs/Center01graph.pickle')
matrix8= nx.convert.to_dict_of_dicts(graph8)
paths=("/content/gdrive/MyDrive/Robbery_Daily_experiment")
graphdf8=pd.read_pickle('/content/gdrive/MyDrive/MainFiles/CenterSide/Graphs/graphCdf.pkl')
def write_to_dict(path,TransactionDF,segmentsDF,matrix_dict):
  import glob
  all_files = glob.glob(paths + "/*.csv")
  mhr_values={}
  for m in all_files:
      df1=neighborhood_transactions(m,graphdf8,matrix8)
      a,b,c=calc_MHR(df1,RobberyTable_daily,graphdf8)
      key=str("mhr"+m.split('/')[-1])
      mhr_values[key]=b
  import csv
  a_file = open('/content/gdrive/MyDrive/MainFiles/Baselines/Robbery/STGCN/WithoutEX/Seed Tuning/Daily/Robbery_daily_best.csv', 'w',newline="")
  writer = csv.writer(a_file)
  for key, value in mhr_values.items():
      writer.writerow([key, value])
  a_file.close()
write_to_dict(paths,RobberyTable_daily,graphdf8,matrix8)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in long_scalars
